In [1]:
import pandas as pd
import sqlite3
import os
import itertools
from tqdm.auto import tqdm
from IPython.display import display

# --- Step 1: 設定 ---
DB_PATH = "../data/processed/s2orc_filtered.db"
# 訓練用に分割されたデータ論文のリスト（150件）
TRAINING_DATAPAPERS_FILE = "../data/datapapers/sampled/training_data_papers_50.csv"
OUTPUT_FILE = "../data/processed/training_dataset_abstract.csv"
NEGATIVE_SAMPLE_RATIO = 4 # 正例1に対する負例の数

def create_training_dataset_abstract():
    """
    人間がアノテーションした結果に基づき、
    アブストラクトを使用した訓練データセットを作成する。
    """
    print("--- Creating Final Training Dataset (Abstract Version) ---")

    if not os.path.exists(TRAINING_DATAPAPERS_FILE):
        print(f"❌ Error: Training data paper file not found.")
        return

    try:
        with sqlite3.connect(DB_PATH) as conn:
            
            # --- 1. 訓練用のデータ論文DOIリストを取得 ---
            df_train_papers = pd.read_csv(TRAINING_DATAPAPERS_FILE)
            train_dois = tuple(df_train_papers['cited_datapaper_doi'].unique())
            
            # --- 2. 正例ペアの元となる論文リストを取得 ---
            placeholders = ','.join('?' for _ in train_dois)
            query = f"""
                SELECT
                    cited_datapaper_doi,
                    citing_doi,
                    human_annotation_status
                FROM
                    positive_candidates
                WHERE
                    cited_datapaper_doi IN ({placeholders})
                    AND (human_annotation_status = 1 OR (llm_annotation_status = 1 AND human_annotation_status = 0))
            """
            df_candidates = pd.read_sql_query(query, conn, params=train_dois)

            # --- 3. 正例ペアの作成 (DOIの組み合わせ) ---
            positive_pairs = []
            grouped = df_candidates.groupby('cited_datapaper_doi')
            
            for data_paper_doi, group in tqdm(grouped, desc="Generating Positive Pairs"):
                human_used_dois = group[group['human_annotation_status'] == 1]['citing_doi'].tolist()
                llm_used_dois = group[group['human_annotation_status'] == 0]['citing_doi'].tolist() # 白
                
                if not human_used_dois:
                    continue 

                # (緑, 緑) ペア
                for pair in itertools.combinations(human_used_dois, 2):
                    positive_pairs.append({
                        'doi_a': pair[0], 
                        'doi_b': pair[1], 
                        'label': 1,
                        'data_paper_doi': data_paper_doi
                    })
                    
                # (緑, 白) ペア
                for human_doi in human_used_dois:
                    for llm_doi in llm_used_dois:
                        positive_pairs.append({
                            'doi_a': human_doi, 
                            'doi_b': llm_doi, 
                            'label': 1,
                            'data_paper_doi': data_paper_doi
                        })

            df_positive = pd.DataFrame(positive_pairs)
            if df_positive.empty:
                print("No positive pairs were generated.")
                return
                
            print(f"Generated {len(df_positive):,} positive pairs.")
            
            # --- 4. 負例ペアの作成 (Easy Negative) ---
            num_negatives_to_sample = len(df_positive) * NEGATIVE_SAMPLE_RATIO
            positive_dois = set(df_positive['doi_a']) | set(df_positive['doi_b'])
            
            # アンカー論文（doi_a）をランダムに選ぶ
            anchor_dois_for_negative = df_positive['doi_a'].sample(n=num_negatives_to_sample, replace=True).tolist()
            
            # DBからランダムに論文をサンプリング
            # `papers`テーブルから直接アブストラクトを取得できる論文を対象にする
            query_random = f"""
                SELECT doi FROM papers 
                WHERE doi NOT IN (SELECT doi FROM full_texts) 
                  AND abstract IS NOT NULL AND abstract != ''
                ORDER BY RANDOM() 
                LIMIT {num_negatives_to_sample * 2}
            """
            df_random_papers = pd.read_sql_query(query_random, conn)
            random_dois = df_random_papers[~df_random_papers['doi'].isin(positive_dois)]['doi'].tolist()
            
            if len(random_dois) < num_negatives_to_sample:
                print(f"Warning: Could only sample {len(random_dois)} negative papers.")
                num_negatives_to_sample = len(random_dois)
            
            negative_pairs = []
            for i in range(num_negatives_to_sample):
                negative_pairs.append({
                    'doi_a': anchor_dois_for_negative[i], 
                    'doi_b': random_dois[i], 
                    'label': 0,
                    'data_paper_doi': None # 負例なので該当なし
                })
            
            df_negative = pd.DataFrame(negative_pairs)
            print(f"Generated {len(df_negative):,} negative pairs.")

            # --- 5. 全ペアの結合とアブストラクトの取得 ---
            df_final_pairs = pd.concat([df_positive, df_negative]).reset_index(drop=True)
            
            all_dois = set(df_final_pairs['doi_a']) | set(df_final_pairs['doi_b'])
            
            print(f"Fetching abstracts for {len(all_dois):,} unique papers...")
            query_texts = f"SELECT doi, abstract FROM papers WHERE doi IN ({','.join('?'*len(all_dois))})"
            df_texts = pd.read_sql_query(query_texts, conn, params=list(all_dois))
            text_map = dict(zip(df_texts['doi'], df_texts['abstract']))

            # --- 6. 最終データセットの作成と保存 ---
            df_final_dataset = pd.DataFrame()
            df_final_dataset['abstract_a'] = df_final_pairs['doi_a'].map(text_map)
            df_final_dataset['abstract_b'] = df_final_pairs['doi_b'].map(text_map)
            df_final_dataset['label'] = df_final_pairs['label']
            df_final_dataset['data_paper_doi'] = df_final_pairs['data_paper_doi']
            
            # アブストラクトが取得できなかった行を削除
            df_final_dataset = df_final_dataset.dropna(subset=['abstract_a', 'abstract_b']).reset_index(drop=True)
            
            print(f"Saving final dataset with {len(df_final_dataset):,} pairs to {OUTPUT_FILE}...")
            df_final_dataset.to_csv(OUTPUT_FILE, index=False)
            
            print("\n--- Process Complete ---")
            display(df_final_dataset.head())

    except Exception as e:
        print(f"💥 An error occurred: {e}")

# --- 実行 ---
create_training_dataset_abstract()

c:\Users\kurokawa\Project\s2orcRanker\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- Creating Final Training Dataset (Abstract Version) ---


Generating Positive Pairs: 100%|██████████| 136/136 [00:00<00:00, 4533.34it/s]


Generated 7,063 positive pairs.
Generated 28,252 negative pairs.
Fetching abstracts for 29,523 unique papers...
Saving final dataset with 35,315 pairs to ../data/processed/training_dataset_abstract.csv...

--- Process Complete ---


,abstract_a,abstract_b,label,data_paper_doi
0,We present climatology and trends of the UTCI ...,The modern unambiguous climate change reveals ...,1,10.1002/GDJ3.102
1,We present climatology and trends of the UTCI ...,Outdoor thermal comfort (OTC) surveys require ...,1,10.1002/GDJ3.102
2,The modern unambiguous climate change reveals ...,Outdoor thermal comfort (OTC) surveys require ...,1,10.1002/GDJ3.102
3,We present climatology and trends of the UTCI ...,"In the months of March-June, India experiences...",1,10.1002/GDJ3.102
4,We present climatology and trends of the UTCI ...,Extreme compound heatwaves (ECHWs) have the po...,1,10.1002/GDJ3.102
